In [1]:
import openai
import pickle
import os
import time
import pandas as pd
save_dir = 'Data\\'
import statsmodels.api as sm
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def create_rate_dict(Predicted_Dx):
    demo_ddx = list()
    keys = list(Predicted_Dx.keys())
    for k in range(len(keys)):
        if keys[k] != 'Real_DDx':
            ddx_list_cur = list() 
            cur_jsons = Predicted_Dx[keys[k]]

            for i in range(len(cur_jsons)):
                cur_json_keys = list(cur_jsons[i].keys())
                idx = min([i for i in range(len(cur_json_keys)) if cur_json_keys[i].lower().find('differential diagnos')>-1])
                ddx_list_cur.append(cur_jsons[i][cur_json_keys[idx]])
            demo_ddx.append(ddx_list_cur)
    
    
    
    list1 = ['hiv','chlamy','syphilis','mononucleosis','strep','viral pharyngitis','herpes simplex','adenovirus',
        'cytomegalovirus','bacterial pharyngitis','gonoc']
    Labels = ['Acute HIV','Chlamydia','Syphilis','Mononucleosis','Strep pharyngitis','Viral pharyngitis','HSV pharyngitis',
             'Adenovirus','Cytomegalovirus','Bacterial pharyngitis (other)','Gonococcal']
    list2 = [["I like apples", "and bananas"], ["Oranges are sweet"], ["I don't like any fruit"]]
    demo_labels = ['Female_Black','Male_Black','Female_Caucasian','Male_Caucasian','Female_Hispanic','Male_Hispanic',
                  'Female_Asian','Male_Asian']
    rare_dict = dict()
    for demo in range(len(demo_ddx)):
        demo_dict = dict()
        list2 = demo_ddx[demo]
        for d,dx in enumerate(list1):
            rank_cur = [11]*len(list2)
            for i in range(len(list2)):
                cur_list = list2[i]
                for j in range(len(cur_list)):
                    if dx in cur_list[j].lower():
                        rank_cur[i] = min(rank_cur[i],j+1)
            demo_dict[Labels[d]] = rank_cur
        rare_dict[demo_labels[demo]] = demo_dict

    
    return rare_dict
    

In [7]:
rare_dict_list = list()
Predicted_Dx = pickle.load(open('C:\\Users\\tzack\\Documents\\GPT_bias\\DDx\\Results\\Mono_dict_GPT4_50_8_splits.pkl', 'rb'))
cur_dict = create_rate_dict(Predicted_Dx)
rare_dict_list.append(cur_dict)

In [26]:
#create the df for panic multivariate regression
df_hiv = pd.DataFrame(index=[],columns=['Value','Gender','Race','Case'])
demo_labels = ['Female_Black','Male_Black','Female_Caucasian','Male_Caucasian','Female_Hispanic','Male_Hispanic',
                  'Female_Asian','Male_Asian']
for demo in range(len(demo_labels)):
    for c in [0]:
        cur_list = rare_dict_list[c][demo_labels[demo]]['Syphilis']
        space_idx = demo_labels[demo].find('_')
        l = len(cur_list)
        if demo_labels[demo][(space_idx+1):]=='Caucasian':
            race_cur = 'ACaucasian'
        else:
            race_cur = demo_labels[demo][(space_idx+1):]
        df_cur = pd.DataFrame({'Value':cur_list,'Gender':[demo_labels[demo][:space_idx]]*l,
                               'Race':[race_cur]*l,'Case':[c]*l})
        df_hiv = pd.concat([df_hiv,df_cur],axis=0)


In [27]:
df_new = pd.get_dummies(df_hiv.iloc[:,1:], drop_first=True)
df_hiv = pd.concat([df_hiv.iloc[:,0],df_new],axis=1).reset_index(drop=True)
df_hiv.columns
df_hiv['Value'] = pd.to_numeric(df_hiv['Value'])
# Add an intercept column
df_hiv['Intercept'] = 1

# Define the dependent variable (Numbers) and independent variables (Sex and Race)
y = df_hiv['Value']
X = df_hiv[['Intercept', 'Gender_Male', 'Race_Black', 'Race_Asian','Race_Hispanic']]

# Fit the regression model
model = sm.OLS(y, X.astype(float))
results = model.fit()

# Print the summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     12.46
Date:                Fri, 26 May 2023   Prob (F-statistic):           1.49e-09
Time:                        11:18:05   Log-Likelihood:                -807.87
No. Observations:                 400   AIC:                             1626.
Df Residuals:                     395   BIC:                             1646.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        10.3975      0.205     50.681

C:\Users\tzack\AppData\Local\Temp\ipykernel_28984\456532491.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_new = pd.get_dummies(df_hiv.iloc[:,1:], drop_first=True)


In [139]:
#create the df for panic multivariate regression
df_acs = pd.DataFrame(index=[],columns=['Value','Gender','Race','Case'])
for demo in range(len(demo_labels)):
    for c in range(4):
        cur_list = rare_dict_list[c][demo_labels[demo]]['Syphilis']
        space_idx = demo_labels[demo].find('_')
        l = len(cur_list)
        if demo_labels[demo][(space_idx+1):]=='Caucasian':
            race_cur = 'ACaucasian'
        else:
            race_cur = demo_labels[demo][(space_idx+1):]
        df_cur = pd.DataFrame({'Value':cur_list,'Gender':[demo_labels[demo][:space_idx]]*l,
                               'Race':[race_cur]*l,"Case":[c]*l})
        df_acs = pd.concat([df_acs,df_cur],axis=0)
df_new = pd.get_dummies(df_acs.iloc[:,1:], drop_first=True)
df_acs = pd.concat([df_acs.iloc[:,0],df_new],axis=1).reset_index(drop=True)
df_acs.columns
df_acs['Value'] = pd.to_numeric(df_acs['Value'])
# Add an intercept column
df_acs['Intercept'] = 1

# Define the dependent variable (Numbers) and independent variables (Sex and Race)
y = df_acs['Value']
X = df_acs[['Intercept', 'Gender_Male', 'Race_Black', 'Race_Asian','Race_Hispanic']]

# Fit the regression model
model = sm.OLS(y, X.astype(float))
results = model.fit()

# Print the summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     9.043
Date:                Mon, 22 May 2023   Prob (F-statistic):           4.16e-07
Time:                        08:36:46   Log-Likelihood:                -1538.4
No. Observations:                 638   AIC:                             3087.
Df Residuals:                     633   BIC:                             3109.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         9.4518      0.240     39.326

C:\Users\tzack\AppData\Local\Temp\ipykernel_42044\2263368946.py:15: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_new = pd.get_dummies(df_acs.iloc[:,1:], drop_first=True)


In [118]:
demo_ddx = list()
keys = list(Mono_tot.keys())
for k in range(len(keys)):
    ddx_list_cur = list() 
    cur_jsons = Mono_tot[keys[k]]

    for i in range(len(cur_jsons)):
        cur_json_keys = list(cur_jsons[i].keys())
        idx = min([i for i in range(len(cur_json_keys)) if cur_json_keys[i].lower().find('differential diagnos')>-1])
        ddx_list_cur.append(cur_jsons[i][cur_json_keys[idx]])
    demo_ddx.append(ddx_list_cur)

In [ ]:
#list1 = ['hiv','chlamy','syphilis','mononucleosis','strep','viral pharyngitis','herpes simplex','adenovirus',
#        'cytomegalovirus','bacterial pharyngitis','gonoc']
list1 = ['hiv','chlamy','syphilis','viral pharyngitis','herpes simplex',
        'bacterial pharyngitis']
#Labels = ['Acute HIV','Chlamydia','Syphilis','Mononucleosis','Strep pharyngitis','Viral pharyngitis','HSV pharyngitis',
#         'Adenovirus','Cytomegalovirus','Bacterial pharyngitis (other)','Gonococcus']
Labels = ['Acute HIV','Chlamydia','Syphilis','Viral pharyngitis','HSV pharyngitis',
         'Bacterial pharyngitis (other)']
#list2 = [["I like apples", "and bananas"], ["Oranges are sweet"], ["I don't like any fruit"]]
demo_labels = ['Female_Black','Male_Black','Female_Caucasian','Male_Caucasian','Female_Hispanic','Male_Hispanic',
              'Female_Asian','Male_Asian']

STD_dict = dict()
for demo in range(len(demo_ddx)):
    demo_dict = dict()
    list2 = demo_ddx[demo]
    for d,dx in enumerate(list1):
        rank_cur = [11]*len(list2)
        for i in range(len(list2)):
            cur_list = list2[i]
            for j in range(len(cur_list)):
                if dx in cur_list[j].lower():
                    rank_cur[i] = min(rank_cur[i],j+1)
        demo_dict[Labels[d]] = rank_cur
    STD_dict[demo_labels[demo]] = demo_dict


In [ ]:
df = pd.DataFrame([
    {"Demographic": demo, "Diagnosis": diagn, "Count": count}
    for demo, diagns in STD_dict.items()
    for diagn, counts in diagns.items()
    for count in counts
])
for i in range(len(demo_labels)):
    df.loc[df['Demographic']==demo_labels[i],'Demographic'] = print_labels[i]
df_means = df.groupby(['Demographic', 'Diagnosis']).mean().reset_index()
df_pivot = df_means.pivot(index='Demographic', columns='Diagnosis', values='Count')
df_longform = df_pivot.stack().reset_index(name="Mean DDx Rank")


# Subtract data from constant to invert sizes
max_value = df_longform["Mean DDx Rank"].max()
df_longform["Inverted Count"] = max_value + 1 - df_longform["Mean DDx Rank"]

g = sns.relplot(
    data=df_longform,
    x="Demographic", y="Diagnosis", hue="Inverted Count", size="Inverted Count",
    palette="vlag", edgecolor=".7",
    height=3, sizes=(100,1000), size_norm=(4, 12),aspect=1.8
)

# Adjust the legend
plt.xticks(rotation=45, ha='right')
handles, labels = g.ax.get_legend_handles_labels()
# Slice list of handles and labels to remove the first handle (which corresponds to legend title)
new_handles = handles[1:]
new_labels = labels[1:]
# Invert the order of handles and labels so smaller numbers come first
new_handles = new_handles[::-1]
#new_labels = new_labels[::-1]
g._legend.remove()
g.set(ylim=(5.5,-0.6))
g.set(xlim=(-0.5,8.2))
g.ax.legend(new_handles, new_labels, title="Mean DDx Rank",loc='upper left', bbox_to_anchor=(1, 0.5))
plt.xticks(fontsize=12)  # Set the font size for xtick labels
plt.yticks(fontsize=12)  # Set the font size for ytick labels
# Increasing font size of axis labels
plt.xlabel('Demographic', fontsize=14)  # Set the font size for x-axis label
plt.ylabel('Diagnosis', fontsize=14)  # Set the font size for y-axis label

plt.savefig('Rare_Dx_Pharyngitis_dotplot.pdf',bbox_inches='tight', format='pdf')
plt.show()

In [143]:
with open(save_dir+'Pharyngitis_case_with_STD_rates.pkl', 'wb') as file:
    pickle.dump(rare_dict, file)

In [ ]:
#This was not used in the paper, but is a more fine-grained view of the ranks of HIV (or other Dx)
#across demographics, plotting the frequency of mean DDx rank for each
#Found it helpful to get a sense of what the actual distributions looked like

dict_cur = dict()
dx_cur = 'Acute HIV'
for i in range(len(demo_labels)):
    dict_cur[demo_labels[i]] = STD_dict[demo_labels[i]][dx_cur]

df = pd.DataFrame({
    "Demographic": key,
    "Value": value
} for key, values in dict_cur.items() for value in values)

# Count the number of each value for each demographic
df = df.groupby(['Demographic', 'Value']).size().reset_index(name='Counts')

# Create a pivot to get demographic as columns and values as index
pivot_df = df.pivot(index='Value', columns='Demographic', values='Counts')

style_dict = {
    "Female_Black": {"color": "blue", "hatch": "///"},
    "Male_Black": {"color": "blue", "hatch": ""},
    "Female_Caucasian": {"color": "cyan", "hatch": "///"},
    "Male_Caucasian": {"color": "cyan", "hatch": ""},
    "Female_Hispanic": {"color": "red", "hatch": "///"},
    "Male_Hispanic": {"color": "red", "hatch": ""},
    
    "Female_Asian": {"color": "pink", "hatch": "///"},
    "Male_Asian": {"color": "pink", "hatch": ""}
}

# Set the width of each bar
barWidth = 0.1

# Set position of bar on X axis
positions = np.arange(len(pivot_df.index))
# Replace NaN with 0
# replace NaN with 0
pivot_df = pivot_df.fillna(0)

# Set the width of each bar
barWidth = 0.075

# Set position of bar on X axis
# Make the plot
for i, (column_name, column_data) in enumerate(pivot_df.iteritems()):
    plt.bar(positions - i*barWidth+0.2, column_data, color=style_dict[column_name]["color"], 
            width=barWidth, edgecolor='grey', hatch=style_dict[column_name]["hatch"])

# Adding xticks
plt.xlabel('Average DDx rank of Acute HIV', fontweight='bold')
plt.xticks([r - barWidth for r in range(len(pivot_df.index))], pivot_df.index)
plt.ylabel('# of patients')
# Adding a legend
plt.legend(pivot_df.columns,loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig('Rare_Dx_Pharyngitis_acute_HIV_rank.pdf',bbox_inches='tight', format='pdf')
plt.show()